<div style="text-align: center;">
  <h1>Cybersecurity Attacks</h1>
</div>

## Contexte
La cybersécurité est devenue un enjeu majeur dans un monde de plus en plus connecté.<br> 
Les incidents de cybersécurité peuvent avoir des impacts graves sur les entreprises, allant de la perte de données à des violations massives de confidentialité.<br>
Une compréhension approfondie des tendances temporelles des attaques et une capacité à prédire leur gravité (faible, moyenne, élevée) sont essentielles pour optimiser les stratégies de défense, renforcer la surveillance pendant les périodes à risque, et prioriser les réponses aux incidents critiques. 

## Source des Données
Ce dataset est disponible sur le dépôt GitHub : https://github.com/incribo-inc/cybersecurity_attacks.<br>  
Il recense divers incidents de cybersécurité en fournissant des informations détaillées sur le trafic réseau, les actions prises, et le contexte associé.<br> 
Les données concernent principalement des incidents observés en Inde, offrant ainsi un aperçu des vulnérabilités spécifiques à cette région.

## Description des Données
Le fichier contient 25 colonnes et un total de 40000 lignes. Ces données permettent de mieux comprendre les schémas des attaques, les segments vulnérables et les réponses associées.

### Variables Principales et Leur Signification
- **Timestamp** : Horodatage de l'événement, indiquant quand l'incident a eu lieu.
- **Source IP Address** et **Destination IP Address** : Adresses IP de la source et de la destination impliquées dans le trafic.
- **Source Port** et **Destination Port** : Ports réseau utilisés par la source et la destination.
- **Protocol** : Protocole de communication (ex : ICMP, UDP, TCP).
- **Packet Length** : Taille des paquets transmis, mesurée en octets.
- **Packet Type** : Type de paquet (données ou contrôle).
- **Traffic Type** : Type de trafic (HTTP, DNS, etc.).
- **Severity Level** : Niveau de gravité de l'incident (Low, Medium, High, etc.).
- **Geo-location Data** : Localisation géographique associée à l'incident.
- **Action Taken** : Action entreprise pour répondre à l'incident (ex : Bloqué, Ignoré, etc.).
- **User Information** et **Device Information** : Informations sur l'utilisateur et le périphérique impliqués.
- **Network Segment** : Segment réseau où l'incident a été enregistré.
- **IDS/IPS Alerts** : Alertes issues de systèmes de détection et de prévention des intrusions.

Le projet aborde les questions suivantes, structurées autour des deux objectifs principaux :
-**Analyse temporelle via clustering** : Regrouper les périodes similaires en fonction des schémas d’attaques pour identifier les moments à haut risque.
-**Prédiction de la gravité** : Développer un modèle de classification pour prédire le niveau de gravité des attaques en fonction des caractéristiques disponibles.


In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import re


## Data Loading

In [2]:
path="../data/raw/cybersecurity_attacks.csv"
data= pd.read_csv(path)
data.head

<bound method NDFrame.head of                  Timestamp Source IP Address Destination IP Address  \
0      2023-05-30 06:33:58     103.216.15.12           84.9.164.252   
1      2020-08-26 07:08:30    78.199.217.198         66.191.137.154   
2      2022-11-13 08:23:25      63.79.210.48          198.219.82.17   
3      2023-07-02 10:38:46     163.42.196.10        101.228.192.255   
4      2023-07-16 13:11:07     71.166.185.76        189.243.174.238   
...                    ...               ...                    ...   
39995  2023-05-26 14:08:42      26.36.109.26         121.100.75.240   
39996  2023-03-27 00:38:27      17.21.163.81         196.108.134.78   
39997  2022-03-31 01:45:49     162.35.217.57            98.107.0.15   
39998  2023-09-22 18:32:38    208.72.233.205         173.79.112.252   
39999  2023-10-10 11:59:52     14.102.21.108           109.198.45.7   

       Source Port  Destination Port Protocol  Packet Length Packet Type  \
0            31225             17616     

## Data Exploration

----------------------

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 25 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Timestamp               40000 non-null  object 
 1   Source IP Address       40000 non-null  object 
 2   Destination IP Address  40000 non-null  object 
 3   Source Port             40000 non-null  int64  
 4   Destination Port        40000 non-null  int64  
 5   Protocol                40000 non-null  object 
 6   Packet Length           40000 non-null  int64  
 7   Packet Type             40000 non-null  object 
 8   Traffic Type            40000 non-null  object 
 9   Payload Data            40000 non-null  object 
 10  Malware Indicators      20000 non-null  object 
 11  Anomaly Scores          40000 non-null  float64
 12  Alerts/Warnings         19933 non-null  object 
 13  Attack Type             40000 non-null  object 
 14  Attack Signature        40000 non-null

### Description du jeu de données

Ce dataset comprend **25 variables** réparties en :

#### Variables catégorielles (17) :
- `Timestamp` (format objet)
- `Source IP Address`
- `Destination IP Address` 
- `Protocol`
- `Packet Type`
- `Traffic Type`
- `Payload Data`
- `Malware Indicators` (50% de valeurs manquantes)
- `Alerts/Warnings` (~50% manquants)
- `Attack Type`
- `Attack Signature`
- `Action Taken`
- `Severity Level` 
- `User Information`
- `Device Information`
- `Network Segment`
- `Geo-location Data`
- `Proxy Information` (~50% manquants)
- `Firewall Logs` (~50% manquants)
- `IDS/IPS Alerts` (~50% manquants)
- `Log Source`

#### Variables numériques (4) :
- `Source Port` (int64)
- `Destination Port` (int64) 
- `Packet Length` (int64)
- `Anomaly Scores` (float64)
---

In [4]:
print("Missing data :")
data.isnull().sum().sort_values(ascending=False)

Missing data :


Alerts/Warnings           20067
IDS/IPS Alerts            20050
Malware Indicators        20000
Firewall Logs             19961
Proxy Information         19851
Attack Type                   0
Geo-location Data             0
Network Segment               0
Device Information            0
User Information              0
Severity Level                0
Action Taken                  0
Attack Signature              0
Timestamp                     0
Source IP Address             0
Anomaly Scores                0
Payload Data                  0
Traffic Type                  0
Packet Type                   0
Packet Length                 0
Protocol                      0
Destination Port              0
Source Port                   0
Destination IP Address        0
Log Source                    0
dtype: int64

## Data Preparation

+ Suppression de la colonne "Payload Data"et"User Information" :<br>
 Ces champs ne semblaient pas nécessaires pour l'analyse, donc on les a éliminé pour alléger le jeu de données et éviter les confusions.<br>
+ Remplacement des valeurs manquantes :<br>
. "Alerts/Warnings" → "Alert Not Triggered" (précise qu'aucune alerte n'a été déclenchée).<br>
. "Firewall Logs" → "No Data" (indique l'absence d'enregistrements).<br>
. "Proxy Information" → "No Data" (standardise les données manquantes).<br>
. "IDS/IPS Alerts" → "No Data" (évite les valeurs vides).<br>
. "Malware Indicators" → "Unknown" (indique une absence ou une incertitude sur les indicateurs de malware).<br>

In [13]:
cleaned_data = data.drop(["Payload Data", "User Information"], axis=1)
cleaned_data["Alerts/Warnings"] = cleaned_data["Alerts/Warnings"].fillna("Alert Not Triggered")
cleaned_data["Firewall Logs"] = cleaned_data["Firewall Logs"].fillna("No Data")
cleaned_data["Proxy Information"] = cleaned_data["Proxy Information"].fillna("No Data")
cleaned_data["IDS/IPS Alerts"] = cleaned_data["IDS/IPS Alerts"].fillna("No Data")
cleaned_data["Malware Indicators"] = cleaned_data["Malware Indicators"].fillna("Unknown")
cleaned_data.isnull().sum().sort_values(ascending=False)


Timestamp                 0
Attack Type               0
IDS/IPS Alerts            0
Firewall Logs             0
Proxy Information         0
Geo-location Data         0
Network Segment           0
Device Information        0
Severity Level            0
Action Taken              0
Attack Signature          0
Alerts/Warnings           0
Source IP Address         0
Anomaly Scores            0
Malware Indicators        0
Traffic Type              0
Packet Type               0
Packet Length             0
Protocol                  0
Destination Port          0
Source Port               0
Destination IP Address    0
Log Source                0
dtype: int64

In [14]:
cleaned_data['Browser'] = cleaned_data['Device Information'].str.split('/').str[0]


In [15]:

platform = r'(Windows|Linux|Android|iPad|iPod|iPhone|Macintosh)'
cleaned_data['Platform'] = cleaned_data['Device Information'].str.extract(platform, flags=re.I).fillna('Unknown')
cleaned_data = cleaned_data.drop('Device Information', axis = 1)

In [16]:
# Extract time features
cleaned_data["Timestamp"] = pd.to_datetime(cleaned_data["Timestamp"], errors='coerce')
cleaned_data['Year'] = cleaned_data["Timestamp"].dt.year
cleaned_data['Month'] = cleaned_data["Timestamp"].dt.month
cleaned_data['Day'] = cleaned_data["Timestamp"].dt.day
cleaned_data['Hour'] = cleaned_data["Timestamp"].dt.hour
cleaned_data['Minute'] = cleaned_data["Timestamp"].dt.minute
cleaned_data['DayOfWeek'] = cleaned_data["Timestamp"].dt.dayofweek
cleaned_data['Hour_Bin'] = pd.cut(cleaned_data['Hour'], bins=[0, 6, 12, 18, 24], labels=['Night', 'Morning', 'Afternoon', 'Evening'])
print(cleaned_data['Hour_Bin'].value_counts())

Hour_Bin
Afternoon    10130
Night         9964
Morning       9870
Evening       8331
Name: count, dtype: int64


In [17]:
variables = [
    'Protocol', 'Packet Type', 'Traffic Type', 
    'Malware Indicators', 'Log Source', 'Browser', 
    'Platform', 'Year'
]

print("*"*50)


for var in variables:
    if var in cleaned_data.columns:
        print(f"\n--- {var} ---")
        
        # Cas spécial pour l'année
        if var == 'Year':
            print(f"Plage : {cleaned_data['Year'].min()} à {cleaned_data['Year'].max()}")
        else:
            # Affiche les 10 valeurs les plus fréquentes
            print(cleaned_data[var].value_counts().head(10))
   

**************************************************

--- Protocol ---
Protocol
ICMP    13429
UDP     13299
TCP     13272
Name: count, dtype: int64

--- Packet Type ---
Packet Type
Control    20237
Data       19763
Name: count, dtype: int64

--- Traffic Type ---
Traffic Type
DNS     13376
HTTP    13360
FTP     13264
Name: count, dtype: int64

--- Malware Indicators ---
Malware Indicators
IoC Detected    20000
Unknown         20000
Name: count, dtype: int64

--- Log Source ---
Log Source
Firewall    20116
Server      19884
Name: count, dtype: int64

--- Browser ---
Browser
Mozilla    31951
Opera       8049
Name: count, dtype: int64

--- Platform ---
Platform
Windows      17953
Linux         8840
Macintosh     5813
iPod          2656
Android       1620
iPhone        1567
iPad          1551
Name: count, dtype: int64

--- Year ---
Plage : 2020 à 2023


In [18]:
new_path="../data/processed/processed_cybersecurity_attacks.csv"
cleaned_data.to_csv(new_path)

---

###  Encodage des variables catégorielles avec OneHotEncoder

Dans cette étape, nous avons appliqué un **encodage One-Hot** aux variables catégorielles du jeu de données.  
L'objectif de cette opération est de convertir les **données non numériques** (comme les protocoles ou les types d'attaques) en un format numérique exploitable par les algorithmes d'analyse tels que le clustering ou la détection d'anomalies.

Nous avons sélectionné les colonnes suivantes pour l'encodage :

- `Protocol`
- `Traffic Type`
- `Severity Level`
- `Action Taken`
- `Packet Type`
- `Network Segment`
- `Platform`
- `Attack Type`
- `Browser`

Ces variables ont été choisies car :
- Elles possèdent un **nombre raisonnable de catégories uniques** (faible cardinalité),
- Elles sont **pertinentes** pour l'analyse comportementale des attaques et des événements réseau,
- Elles ne sont **ni ordinales** (pas d’ordre logique entre les valeurs),
- Elles peuvent avoir une influence sur la **segmentation des attaques** ou la **détection d’anomalies**.


In [19]:
from sklearn.preprocessing import OneHotEncoder


df = pd.read_csv(new_path)

# Colonnes à encoder
cols_to_encode = ['Protocol', 'Traffic Type', 'Severity Level', 'Action Taken', 'Packet Type', 'Network Segment','Platform', 'Attack Type', 'Browser']

# Appliquer le OneHotEncoder
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded = encoder.fit_transform(df[cols_to_encode])

# Créer un DataFrame encodé
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(cols_to_encode))

# Concaténer avec le dataset d'origine sans les anciennes colonnes
df_encoded = pd.concat([df.drop(columns=cols_to_encode), encoded_df], axis=1)

encoded="../data/processed/encoded_cybersecurity_attacks.csv"
df_encoded.to_csv(encoded)